In [17]:
import pandas as pd
import numpy as np
from glob import glob
import math
import scipy

#from DataTransformation import LowPassFilter , PrincipalComponentAnalysis
#from TemporalAbstraction import NumericalAbstraction
#from FrequencyAbstraction import FourierTransformation
from sklearn.cluster import KMeans
from sklearn.neighbors import LocalOutlierFactor  # pip install scikit-learn


from sklearn.model_selection import train_test_split

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import cross_val_score , RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from scipy.signal import argrelextrema


import pickle

import warnings
warnings.filterwarnings("ignore")

In [3]:
##############################################################
#                                                            #
#    Mark Hoogendoorn and Burkhardt Funk (2017)              #
#    Machine Learning for the Quantified Self                #
#    Springer                                                #
#    Chapter 3                                               #
#                                                            #
##############################################################

# Updated by Dave Ebbelaar on 22-12-2022

from sklearn.decomposition import PCA
from scipy.signal import butter, lfilter, filtfilt
import copy
import pandas as pd

# This class removes the high frequency data (that might be considered noise) from the data.
# We can only apply this when we do not have missing values (i.e. NaN).
class LowPassFilter:
    def low_pass_filter(
        self,
        data_table,
        col,
        sampling_frequency,
        cutoff_frequency,
        order=5,
        phase_shift=True,
    ):
        # http://stackoverflow.com/questions/12093594/how-to-implement-band-pass-butterworth-filter-with-scipy-signal-butter
        # Cutoff frequencies are expressed as the fraction of the Nyquist frequency, which is half the sampling frequency
        nyq = 0.5 * sampling_frequency
        cut = cutoff_frequency / nyq

        b, a = butter(order, cut, btype="low", output="ba", analog=False)
        if phase_shift:
            data_table[col + "_lowpass"] = filtfilt(b, a, data_table[col])
        else:
            data_table[col + "_lowpass"] = lfilter(b, a, data_table[col])
        return data_table


# Class for Principal Component Analysis. We can only apply this when we do not have missing values (i.e. NaN).
# For this we have to impute these first, be aware of this.
class PrincipalComponentAnalysis:

    pca = []

    def __init__(self):
        self.pca = []

    def normalize_dataset(self, data_table, columns):
        dt_norm = copy.deepcopy(data_table)
        for col in columns:
            dt_norm[col] = (data_table[col] - data_table[col].mean()) / (
                data_table[col].max()
                - data_table[col].min()
                # data_table[col].std()
            )
        return dt_norm

    # Perform the PCA on the selected columns and return the explained variance.
    def determine_pc_explained_variance(self, data_table, cols):

        # Normalize the data first.
        dt_norm = self.normalize_dataset(data_table, cols)

        # perform the PCA.
        self.pca = PCA(n_components=len(cols))
        self.pca.fit(dt_norm[cols])
        # And return the explained variances.
        return self.pca.explained_variance_ratio_

    # Apply a PCA given the number of components we have selected.
    # We add new pca columns.
    def apply_pca(self, data_table, cols, number_comp):

        # Normalize the data first.
        dt_norm = self.normalize_dataset(data_table, cols)

        # perform the PCA.
        self.pca = PCA(n_components=number_comp)
        self.pca.fit(dt_norm[cols])

        # Transform our old values.
        new_values = self.pca.transform(dt_norm[cols])

        # And add the new ones:
        for comp in range(0, number_comp):
            data_table["pca_" + str(comp + 1)] = new_values[:, comp]

        return data_table


In [4]:
##############################################################
#                                                            #
#    Mark Hoogendoorn and Burkhardt Funk (2017)              #
#    Machine Learning for the Quantified Self                #
#    Springer                                                #
#    Chapter 4                                               #
#                                                            #
##############################################################

# Updated by Dave Ebbelaar on 22-12-2022

import numpy as np
import scipy.stats as stats

# Class to abstract a history of numerical values we can use as an attribute.
class NumericalAbstraction:

    # This function aggregates a list of values using the specified aggregation
    # function (which can be 'mean', 'max', 'min', 'median', 'std')
    def aggregate_value(self, aggregation_function):
        # Compute the values and return the result.
        if aggregation_function == "mean":
            return np.mean
        elif aggregation_function == "max":
            return np.max
        elif aggregation_function == "min":
            return np.min
        elif aggregation_function == "median":
            return np.median
        elif aggregation_function == "std":
            return np.std
        else:
            return np.nan

    # Abstract numerical columns specified given a window size (i.e. the number of time points from
    # the past considered) and an aggregation function.
    def abstract_numerical(self, data_table, cols, window_size, aggregation_function):

        # Create new columns for the temporal data, pass over the dataset and compute values
        for col in cols:
            data_table[
                col + "_temp_" + aggregation_function + "_ws_" + str(window_size)
            ] = (
                data_table[col]
                .rolling(window_size)
                .apply(self.aggregate_value(aggregation_function))
            )

        return data_table


In [5]:
##############################################################
#                                                            #
#    Mark Hoogendoorn and Burkhardt Funk (2017)              #
#    Machine Learning for the Quantified Self                #
#    Springer                                                #
#    Chapter 4                                               #
#                                                            #
##############################################################

# Updated by Dave Ebbelaar on 06-01-2023

import numpy as np


# This class performs a Fourier transformation on the data to find frequencies that occur
# often and filter noise.
class FourierTransformation:

    # Find the amplitudes of the different frequencies using a fast fourier transformation. Here,
    # the sampling rate expresses the number of samples per second (i.e. Frequency is Hertz of the dataset).
    def find_fft_transformation(self, data, sampling_rate):
        # Create the transformation, this includes the amplitudes of both the real
        # and imaginary part.
        transformation = np.fft.rfft(data, len(data))
        return transformation.real, transformation.imag

    # Get frequencies over a certain window.
    def abstract_frequency(self, data_table, cols, window_size, sampling_rate):

        # Create new columns for the frequency data.
        freqs = np.round((np.fft.rfftfreq(int(window_size)) * sampling_rate), 3)

        for col in cols:
            data_table[col + "_max_freq"] = np.nan
            data_table[col + "_freq_weighted"] = np.nan
            data_table[col + "_pse"] = np.nan
            for freq in freqs:
                data_table[
                    col + "_freq_" + str(freq) + "_Hz_ws_" + str(window_size)
                ] = np.nan

        # Pass over the dataset (we cannot compute it when we do not have enough history)
        # and compute the values.
        for i in range(window_size, len(data_table.index)):
            for col in cols:
                real_ampl, imag_ampl = self.find_fft_transformation(
                    data_table[col].iloc[
                        i - window_size : min(i + 1, len(data_table.index))
                    ],
                    sampling_rate,
                )
                # We only look at the real part in this implementation.
                for j in range(0, len(freqs)):
                    data_table.loc[
                        i, col + "_freq_" + str(freqs[j]) + "_Hz_ws_" + str(window_size)
                    ] = real_ampl[j]
                # And select the dominant frequency. We only consider the positive frequencies for now.

                data_table.loc[i, col + "_max_freq"] = freqs[
                    np.argmax(real_ampl[0 : len(real_ampl)])
                ]
                data_table.loc[i, col + "_freq_weighted"] = float(
                    np.sum(freqs * real_ampl)
                ) / np.sum(real_ampl)
                PSD = np.divide(np.square(real_ampl), float(len(real_ampl)))
                PSD_pdf = np.divide(PSD, np.sum(PSD))
                data_table.loc[i, col + "_pse"] = -np.sum(np.log(PSD_pdf) * PSD_pdf)

        return data_table


In [24]:

class Tracker:
    def __init__(self, files_path):
        self.files = glob(files_path)

    def read_data_from_files(self):
        acc_df = pd.DataFrame()
        gyr_df = pd.DataFrame()

        acc_set = 1
        gyr_set = 1

        for f in self.files:
            label = f.split("-")[1]

            df = pd.read_csv(f)

            df["label"] = label
            
            if "Accelerometer" in f:
                df["set"] = acc_set
                acc_set+= 1
                acc_df = pd.concat([acc_df, df])
            else:
                df["set"] = gyr_set
                gyr_set+= 1
                gyr_df = pd.concat([gyr_df, df])

        acc_df.index = pd.to_datetime(acc_df["epoch (ms)"], unit="ms")
        gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"], unit="ms")

        del acc_df["epoch (ms)"]
        del acc_df["time (01:00)"]
        del acc_df["elapsed (s)"]

        del gyr_df["epoch (ms)"]
        del gyr_df["time (01:00)"]
        del gyr_df["elapsed (s)"]

        return acc_df, gyr_df

    def merge_and_clean_data(self):
        
        acc_df, gyr_df = self.read_data_from_files()
        
        data_merged = pd.concat([acc_df.iloc[:, :3], gyr_df], axis=1)

        data_merged.columns = [
            "acc_x",
            "acc_y",
            "acc_z",
            "gyr_x",
            "gyr_y",
            "gyr_z",
            "label",
            "set",
            
        ]
        
        sampling = {
            'acc_x':"mean",
            'acc_y':"mean",
            'acc_z':"mean",
            'gyr_x':"mean",
            'gyr_y':"mean",
            'gyr_z':"mean",
            'label':"last",
            'set':"last",
            
            
        }

        # Resampling the first 100 data points to a 200ms frequency and applying the defined sampling methods.
        # This is done to illustrate the resampling process.
        data_merged[:100].resample(rule="200ms").apply(sampling)
        days = [g for n,g in data_merged.groupby(pd.Grouper(freq="D"))]
        data_resampled = pd.concat([df.resample(rule="200ms").apply(sampling).dropna() for df in days])
        #del data_resampled["set"]
       
        return data_resampled
    
    
    

    def remove_outliers(self):
        # Insert Chauvenet's function
        def mark_outliers_chauvenet(dataset, col, C=2):
            """Finds outliers in the specified column of datatable and adds a binary column with
            the same name extended with '_outlier' that expresses the result per data point.
            
            Taken from: https://github.com/mhoogen/ML4QS/blob/master/Python3Code/Chapter3/OutlierDetection.py

            Args:
                dataset (pd.DataFrame): The dataset
                col (string): The column you want apply outlier detection to
                C (int, optional): Degree of certainty for the identification of outliers given the assumption 
                                of a normal distribution, typicaly between 1 - 10. Defaults to 2.

            Returns:
                pd.DataFrame: The original dataframe with an extra boolean column 
                indicating whether the value is an outlier or not.
            """

            dataset = dataset.copy()
            # Compute the mean and standard deviation.
            mean = dataset[col].mean()
            std = dataset[col].std()
            N = len(dataset.index)
            criterion = 1.0 / (C * N)

            # Consider the deviation for the data points.
            deviation = abs(dataset[col] - mean) / std

            # Express the upper and lower bounds.
            low = -deviation / math.sqrt(C)
            high = deviation / math.sqrt(C)
            prob = []
            mask = []

            # Pass all rows in the dataset.
            for i in range(0, len(dataset.index)):
                # Determine the probability of observing the point
                prob.append(
                    1.0 - 0.5 * (scipy.special.erf(high[i]) - scipy.special.erf(low[i]))
                )
                # And mark as an outlier when the probability is below our criterion.
                mask.append(prob[i] < criterion)
            dataset[col + "_outlier"] = mask
            return dataset

        df = self.merge_and_clean_data()
        oultier_cols = list(df.columns[:6])
        
        chauvenet_oultier_removed_df = df.copy()

        # Iterate through outlier columns
        for col in oultier_cols:
            # Iterate through unique labels
            for label in df["label"].unique():
                # Mark outliers using Chauvenet's criterion
                dataset = mark_outliers_chauvenet(df[df["label"] == label],col)
                # Set outliers to NaN
                dataset.loc[dataset[col + "_outlier"], col] = np.nan
                # Update the outlier-removed DataFrame
                chauvenet_oultier_removed_df.loc[(chauvenet_oultier_removed_df["label"] == label ), col] = dataset[col]
            
        return chauvenet_oultier_removed_df
    
    
    def low_pass(self):
        df = self.remove_outliers()
        predictor_columns = list(df.columns[:6])
        # We try removing all NaN values and filling gaps with the default method, which is linear interpolation. 
        # For each predictor column, we perform interpolation to fill the missing values. 
        # By default, interpolation computes the mean between the next and last available values in the same row.
        for i in predictor_columns:
            df[i] = df[i].interpolate()
            
        lowpass_df = df.copy()
        LowPass = LowPassFilter()
        
        freq_sample  = 1000 / 200 
        cutoff = 1.3 
        # Apply the lowpass filter to all predictor columns
        for col in predictor_columns:
            lowpass_df = LowPass.low_pass_filter(lowpass_df, col, freq_sample , cutoff, order=5)
            lowpass_df[col] = lowpass_df[col + "_lowpass"]
            del lowpass_df[col + "_lowpass"]
        
        return lowpass_df
    
    def pca(self):
        pca_df = self.low_pass().copy()
        predictor_columns = list(pca_df.columns[:6])
        pca = PrincipalComponentAnalysis()
        pca_df = pca.apply_pca(pca_df, predictor_columns, 3)
        return pca_df
    
    def sum_square(self):
        squares_df = self.pca().copy()
        # Calculate the squared magnitude for accelerometer and gyroscope readings
        acc_r = squares_df['acc_x']**2 + squares_df['acc_y']**2 + squares_df['acc_z']**2 
        gyr_r = squares_df['gyr_x']**2 + squares_df['gyr_y']**2 + squares_df['gyr_z']**2 

        # Calculate the magnitude by taking the square root of the sum of squared components
        squares_df['acc_r'] = np.sqrt(acc_r)
        squares_df['gyr_r'] = np.sqrt(gyr_r)
        return squares_df
    
    def temporal_abstraction(self):
        temporal_df = self.sum_square().copy()
        predictor_columns = list(temporal_df.columns[:6])
        # Add magnitude columns 'acc_r' and 'gyr_r' to the predictor columns list
        predictor_columns = predictor_columns + ['acc_r', 'gyr_r']

        # Instantiate the NumericalAbstraction class
        NumAbs = NumericalAbstraction()

        # Initialize a list to store temporally abstracted subsets
        temporal_df_list = []

        # Iterate over unique sets in the DataFrame
        for set_id in temporal_df['set'].unique():
            # Select subset corresponding to the current set
            subset = temporal_df[temporal_df['set'] == set_id].copy()
            
            # Perform temporal abstraction for each predictor column
            for col in predictor_columns:
                # Calculate the mean and standard deviation with a window size of 5
                subset = NumAbs.abstract_numerical(subset, predictor_columns, window_size=5, aggregation_function='mean')
                subset = NumAbs.abstract_numerical(subset, predictor_columns, window_size=5, aggregation_function='std')

            # Append the abstracted subset to the list
            temporal_df_list.append(subset)

        # Concatenate all abstracted subsets into a single DataFrame
        temporal_df = pd.concat(temporal_df_list)
        
        return temporal_df
    
    def frequency(self):
        frequency_df = self.temporal_abstraction().copy().reset_index()
        predictor_columns = list(frequency_df.columns[:6])
        predictor_columns = predictor_columns + ['acc_r', 'gyr_r']
        # Instantiate the FourierTransformation class
        FreqAbs = FourierTransformation()

        # Define the sampling frequency (fs) and window size (ws)
        fs = int(1000 / 200)  # Sampling frequency (samples per second)
        ws = int(2800 / 200)   # Window size (number of samples)

        # Initialize a list to store Fourier-transformed subsets
        frequency_df_list = []

        # Iterate over unique sets in the DataFrame
        for s in frequency_df["set"].unique():
            # Select subset corresponding to the current set
            subset = frequency_df[frequency_df["set"] == s].reset_index(drop=True).copy()
            # Apply Fourier transformations to predictor columns
            subset = FreqAbs.abstract_frequency(subset, predictor_columns, ws, fs)
            # Append the transformed subset to the list
            frequency_df_list.append(subset)

        # Concatenate all transformed subsets into a single DataFrame and set index
        frequency_df = pd.concat(frequency_df_list).set_index("epoch (ms)", drop=True)

        # --------------------------------------------------------------
        # Dealing with overlapping windows
        # --------------------------------------------------------------
        # Remove rows with any NaN values from the DataFrame
        frequency_df = frequency_df.dropna()

        # Select every other row in the DataFrame
        frequency_df = frequency_df.iloc[::2]
        
        # List of column names to drop
        columns_to_drop = ['epoch (ms)_max_freq', 'epoch (ms)_freq_weighted', 'epoch (ms)_pse',
                        'epoch (ms)_freq_0.0_Hz_ws_14', 'epoch (ms)_freq_0.357_Hz_ws_14',
                        'epoch (ms)_freq_0.714_Hz_ws_14', 'epoch (ms)_freq_1.071_Hz_ws_14',
                        'epoch (ms)_freq_1.429_Hz_ws_14', 'epoch (ms)_freq_1.786_Hz_ws_14',
                        'epoch (ms)_freq_2.143_Hz_ws_14', 'epoch (ms)_freq_2.5_Hz_ws_14']

        # Drop the specified columns
        frequency_df = frequency_df.drop(columns=columns_to_drop)
        
        return frequency_df
    
    def clusters(self):
        cluster_df = self.frequency().copy()
        # Perform KMeans clustering with optimal number of clusters (e.g., 5)
        cluster_columns = ['acc_x', 'acc_y', 'acc_z']
        kmeans = KMeans(n_clusters=5, n_init=20, random_state=0)
        subset = cluster_df[cluster_columns]
        # Predict cluster labels and assign them to the DataFrame
        cluster_df["cluster_label"] = kmeans.fit_predict(subset)
                    
        return cluster_df
    
    def model(self):
        selected_features = ['acc_y_freq_0.0_Hz_ws_14',
                            'acc_z_freq_0.0_Hz_ws_14',
                            'pca_1',
                            'acc_y_temp_mean_ws_5',
                            'cluster_label',
                            'acc_y',
                            'gyr_r_freq_0.0_Hz_ws_14',
                            'pca_2',
                            'acc_z_temp_mean_ws_5',
                            'acc_x_freq_0.0_Hz_ws_14',
                            'acc_z',
                            'acc_x_temp_mean_ws_5',
                            'gyr_z_temp_std_ws_5',
                            'gyr_r_temp_mean_ws_5',
                            'acc_y_max_freq']
        
        input_data = self.clusters()[selected_features]
        
        with open("../models/04_Model.pkl", "rb") as f:
            model = pickle.load(f)
        
        input_values = input_data.to_numpy().reshape(1, -1)
        return model.predict(input_values)
    
    def count_rep(self):
        
        exercise = self.model()[0]
        df = self.merge_and_clean_data()
        
        fs = int(1000 / 200)
        
        column = "acc_r"
        cutoff = 0.4
        
        if exercise == "squat":
            cutoff = 0.34
        
        elif exercise == "row":
            cutoff = 0.65
            column = 'gyr_x'
        
        elif exercise == "ohp":
            cutoff = 0.35
            
        def count_reps(dataset ,cutoff = 0.4 ,order = 10 ,column = column):
            LowPass = LowPassFilter()
            data = LowPass.low_pass_filter(data_table = dataset,col = column,
                                        sampling_frequency = fs,
                                        cutoff_frequency = cutoff,order = order)


            index = argrelextrema(data[column + "_lowpass"].values, np.greater)
            peaks = data.iloc[index]
            return len(peaks)
        
        reps = count_reps(df, cutoff=cutoff, column=column)
        
        return reps
        
        
        
        

In [26]:
tracker = Tracker("../data/raw/MetaMotion/MetaMotion/*.csv")
df = tracker.clusters()



In [27]:
print(tracker.count_rep())

ValueError: X has 58020 features, but RandomForestClassifier is expecting 15 features as input.

In [20]:
selected_features = ['acc_y_freq_0.0_Hz_ws_14',
                            'acc_z_freq_0.0_Hz_ws_14',
                            'pca_1',
                            'acc_y_temp_mean_ws_5',
                            'cluster_label',
                            'acc_y',
                            'gyr_r_freq_0.0_Hz_ws_14',
                            'pca_2',
                            'acc_z_temp_mean_ws_5',
                            'acc_x_freq_0.0_Hz_ws_14',
                            'acc_z',
                            'acc_x_temp_mean_ws_5',
                            'gyr_z_temp_std_ws_5',
                            'gyr_r_temp_mean_ws_5',
                            'acc_y_max_freq']
values = df[selected_features]
input_data = values.iloc[0].to_numpy().reshape(1, -1)
print(input_data)

[[ 1.45027497e+01 -2.07567846e+00 -3.11963218e-01  1.01146254e+00
   1.00000000e+00  9.09032617e-01  2.41915837e+02 -1.67364387e-01
  -1.61931081e-01 -1.69685551e+00 -1.68556248e-01 -1.95715115e-01
   1.47236123e+01  2.28002285e+01  0.00000000e+00]]


In [21]:

with open("../models/04_Model.pkl", "rb") as f:
    model = pickle.load(f)

model.predict(input_data)

array(['bench'], dtype=object)